In [1]:
from ib.ext.Contract import Contract
from ib.ext.ScannerSubscription import ScannerSubscription
from ib.ext.ContractDetails import ContractDetails
from ib.ext.Order import Order
from ib.opt import Connection, message

from time import sleep 
from datetime import datetime
import pandas as pd
# import backtrader as bt
# import pytz, tzlocal
from bs4 import BeautifulSoup
import re
from ibapi.ticktype import TickTypeEnum
import talib

In [2]:
def error_handler(msg):
    """Handles the capturing of error messages"""
    print ("Server Error: {}".format(msg))

def reply_handler(msg):
    """Handles of server replies"""
    print ("Server Response: {}, {}".format(msg.typeName, msg))

def next_id(msg):
    global order_id
    order_id = int(re.findall('\d+', str(msg))[0])
    
    return order_id
    
def historical_data_handler(msg):  
    global historical_data
    
    if ('finished' in str(msg.date)) == False:   
        
        historical_data.loc[len(historical_data)] = ([msg.date,
                                                      msg.open, msg.high, msg.low, msg.close, msg.volume])
    else:  
        historical_data.set_index('Date',inplace=True)

        
def ReportsFinStatements(msg):
    global annual
    global quarter
   
    parser = BeautifulSoup(msg.data, 'lxml')

    mapp = dict()
    for item in parser.findAll('mapitem'):
        mapp[item['coaitem']] = item.text
        
    
    #ANNUAL
    index = [];columns = []
    for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
        index.append(datetime.strptime(item['fiscalyear'], '%Y').year)
    
    for i in parser.findAll('annualperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
        columns.append(mapp[i['coacode']])

    annual = pd.DataFrame(index = index, columns = columns)
    annual.index.name = 'Annual'
    for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
        for i in item.findAll('lineitem'):
            try:
                annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = float(i.text)
            except:
                annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = np.nan
        
    # QUARTER
    index_q = [];columns_q = []
    for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
        index_q.append(datetime.strptime(item['enddate'], '%Y-%m-%d').date())
    
    for i in parser.findAll('interimperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
        columns_q.append(mapp[i['coacode']])

    quarter = pd.DataFrame(index = index_q, columns = columns_q)
    quarter.index.name = 'Quarter'
    for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
        for i in item.findAll('lineitem'):
            try:
                quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = float(i.text)
            except:
                quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = np.nan


def position_account(msg):
    global account
    account = msg

def Scanner(msg):
    global symbol
    global sectype
    global exchange
    global currency
        
    symbol.append(msg.contractDetails.m_summary.m_symbol)
    sectype.append(msg.contractDetails.m_summary.m_secType)
    exchange.append(msg.contractDetails.m_summary.m_exchange)
    currency.append(msg.contractDetails.m_summary.m_currency)

def Analyst_Estimates(msg):
    global resc
    global resc_ann
    global resc_q
    
    parser = BeautifulSoup(msg.data, 'lxml')
    resc.iloc[:,0] = parser.find('name').text    
    resc.iloc[:,1] = parser.find('exchange').text
    resc.iloc[:,2] = parser.findAll('secid')[0].text
    resc.iloc[:,3] = parser.findAll('sector')[0].text

    resc.iloc[:,4] = parser.findAll('marketdataitem')[0].text
    resc.iloc[:,5] = parser.findAll('marketdataitem')[1].text
    resc.iloc[:,6] = parser.findAll('marketdataitem')[2].text
    resc.iloc[:,7] = parser.findAll('marketdataitem')[3].text
    resc.iloc[:,8] = parser.findAll('marketdataitem')[4].text
    resc.iloc[:,9] = parser.findAll('fyactual')[0].text
    
    annual = [];columns = [];quarter = []
    for item in parser.findAll('fyactual'):
        columns.append(item['type'])
        for per in item.findAll('fyperiod'):
            if per['periodtype'] == 'A':
                annual.append(per['fyear'])
            if per['periodtype'] == 'Q':
                quarter.append('{}-{}'.format(per['endcalyear'],per['endmonth']))
    
    index = list(set(annual))
    index_q = list(set(quarter))
    resc_ann = pd.DataFrame(index = index, columns = columns).sort_index(axis = 0,ascending = False)
    resc_q = pd.DataFrame(index = index_q, columns = columns).sort_index(axis = 0,ascending = False)
    
    for item in parser.findAll('fyactual'):
        for per in item.findAll('fyperiod'):
            
            if per['periodtype'] == 'Q':
                try:
                    resc_q.loc['{}-{}'.format(per['endcalyear'],per['endmonth']),item['type']] = float(per.find('actvalue').text)
                except:
                    resc_q.loc['{}-{}'.format(per['endcalyear'],per['endmonth']),item['type']] = np.nan  
                    
            if per['periodtype'] == 'A':
                try:
                    resc_ann.loc[per['fyear'],item['type']] = float(per.find('actvalue').text)
                except:
                    resc_ann.loc[per['fyear'],item['type']] = np.nan

                    
def ReportsFinSummary(msg):
    global fin
    
    parser = BeautifulSoup(msg.data, 'lxml')
    date_div = [];data_div = []
    
    for i in parser.findAll('dividendpershare'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            
            date_div.append(i['asofdate'])
            try:
                data_div.append(float(i.text))
            except:
                data_div.append(np.nan)
                
    data_rev = []
    for i in parser.findAll('totalrevenue'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            try:
                data_rev.append(float(i.text))
            except:
                data_rev.append(np.nan)
                
    data_eps = []
    for i in parser.findAll('eps'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            try:
                data_eps.append(float(i.text))
            except:
                data_eps.append(np.nan)

    fin = pd.DataFrame(index = date_div,
                       data = {'Dividend Per Share(TTM)':data_div,
                               'Total Revenue(TTM)':data_rev, 
                               'EPS(TTM)':data_eps}).sort_index(axis = 0,ascending = False)

def ReportSnapshot(msg):
    global business_summary
    global brief
    global snap
    global address
    global ratio
    global estimate
    
    parser = BeautifulSoup(msg.data, 'lxml')
    
    business_summary = parser.findAll('text')[0].text
    brief = parser.findAll('text')[1].text
    address = '{},{},{},{}'.format(parser.find('contactinfo').find('streetaddress').text,
                                   parser.find('contactinfo').find('city').text,
                                   parser.find('contactinfo').find('state-region').text,
                                   parser.find('contactinfo').find('country').text)
    
    snap = pd.DataFrame(index = [parser.findAll('coid')[1].text],
                        data = {'Company Type': parser.find('cotype').text,
                                'Desc':parser.findAll('issue')[0]['desc'],
                                'Exchange': parser.findAll('exchange')[0].text,
                                'Industry':parser.findAll('industry')[0].text,
                                'Index':parser.find('indexconstituet').text})
    dicted = {};dicted_est = {}
    for i in range(len(parser.find('ratios').findAll('ratio'))):
        try:
            dicted[parser.find('ratios').findAll('ratio')[i]['fieldname']] = float(parser.find('ratios').findAll('ratio')[i].text)
        except:
            dicted[parser.find('ratios').findAll('ratio')[i]['fieldname']] = parser.find('ratios').findAll('ratio')[i].text
            
    ratio = pd.DataFrame(index = [parser.findAll('coid')[1].text], data = dicted)
    
    for i in range(len(parser.find('forecastdata').findAll('ratio'))):
        try:
            dicted_est[parser.find('forecastdata').findAll('ratio')[i]['fieldname']] = float(parser.find('forecastdata').findAll('ratio')[i].text)
        except:
            dicted_est[parser.find('forecastdata').findAll('ratio')[i]['fieldname']] = parser.find('forecastdata').findAll('ratio')[i].text
            
    estimate = pd.DataFrame(index = [parser.findAll('coid')[1].text], data = dicted_est)
    
def create_contract(symbol, sec_type, exch, prim_exch, curr):

    contract = Contract()
    contract.m_symbol = symbol
    contract.m_secType = sec_type
    contract.m_exchange = exch
    contract.m_primaryExch = prim_exch
    contract.m_currency = curr
    
    return contract

def create_order(order_type, quantity, action):

    order = Order()
    order.m_orderType = order_type
    order.m_totalQuantity = quantity
    order.m_action = action
    
    return order

def create_scanner(numberOfRows, instrument, locationCode, ScanCode, 
                   AbovePrice, marketCapAbove, AboveVolume):

    scanSub = ScannerSubscription()
    scanSub.numberOfRows(numberOfRows)
    scanSub.m_instrument = instrument
    scanSub.m_locationCode = locationCode
    scanSub.m_scanCode = ScanCode
    scanSub.m_abovePrice = AbovePrice
    scanSub.m_marketCapAbove = marketCapAbove
    scanSub.m_aboveVolume = AboveVolume
    
    return scanSub



In [3]:
#price = pd.DataFrame(columns = ['Ticker Id','Field','Price'])
#size = pd.DataFrame()
#scan_param = None
#tws_conn.register(ScannerParameters, message.scannerParameters)
#tws_conn.register(mktdata, message.tickPrice)
#tws_conn.reqMarketDataType(4) # switch to delayed frozen data if live is not available
#tws_conn.reqMktData(0,contract,"",True)                    
#tws_conn.reqScannerParameters()

# Connecting to TWS

In [4]:
tws_conn = Connection.create(host='127.0.0.1',port=7497, clientId=0)
tws_conn.connect()

Server Version: 76
TWS Time at connection:20191023 08:59:02 EST


True

In [5]:
tws_conn.register(error_handler, 'Error')
tws_conn.register(reply_handler)
#tws_conn.unregister(error_handler, 'Error')
#tws_conn.unregister(reply_handler)

False

# Step 1 : Filtertation

In [6]:
symbol = [];sectype = [];exchange = []; currency = []

In [7]:
ScanCode = 'LOW_VS_52W_HL'
scanSub = create_scanner(numberOfRows = 10, instrument = 'STK', 
                         locationCode = 'STK.US.MAJOR', ScanCode = ScanCode, 
                         AbovePrice = '100',marketCapAbove = '100000',
                        AboveVolume = '1000')

In [8]:
tws_conn.register(Scanner, message.scannerData)
tws_conn.reqScannerSubscription(0,scanSub)
sleep(5)
tws_conn.unregister(Scanner,message.scannerData)

Server Error: <error id=0, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>


True

In [9]:
scanner = pd.DataFrame()
scanner['Symbol'] = symbol
scanner['SecType'] = sectype
scanner['Exchange'] = exchange
scanner['Currency'] = currency
scanner.index.name = ScanCode 

In [10]:
scanner

,Symbol,SecType,Exchange,Currency
LOW_VS_52W_HL,,,,


# Step 2 : Iterate over selceted tickers and collect data

## ReportsFinStatements: Financial statements

In [11]:
contract = create_contract(symbol = 'FB',
                           sec_type = 'STK',
                           exch = 'SMART',
                           prim_exch = 'NASDAQ',
                           curr = 'USD')

In [12]:
annual = pd.DataFrame()
quarter = pd.DataFrame()

tws_conn.register(ReportsFinStatements, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='ReportsFinStatements')
sleep(5)
tws_conn.unregister(ReportsFinStatements, message.fundamentalData)

Server Error: <error id=0, errorCode=430, errorMsg=We are sorry, but fundamentals data for the security specified is not available.Not allowed>


True

## RESC: Analyst estimates

In [13]:
resc = pd.DataFrame(index = [0],columns = ['Name', 'Exchange', 'Symbol', 'Sector', 'CLPRICE', 'SHARESOUT',
                                          'MARKETCAP', '52WKHIGH', '52WKLOW','NAV'])

resc_ann = pd.DataFrame()
resc_q = pd.DataFrame()

tws_conn.register(Analyst_Estimates, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='RESC')
sleep(5)
tws_conn.unregister(Analyst_Estimates, message.fundamentalData)

Server Error: <error id=0, errorCode=430, errorMsg=We are sorry, but fundamentals data for the security specified is not available.Not allowed>


True

## ReportsFinSummary: Financial summary

In [14]:
fin = None
tws_conn.register(ReportsFinSummary, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='ReportsFinSummary')
sleep(5)
tws_conn.unregister(ReportsFinSummary, message.fundamentalData)

23-Oct-19 08:59:22 ERROR     Exception in message dispatch.  Handler 'ReportsFinSummary' for 'fundamentalData'
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ib/opt/dispatcher.py", line 44, in __call__
    results.append(listener(message))
  File "<ipython-input-2-2ec9e9594457>", line 169, in ReportsFinSummary
    'EPS(TTM)':data_eps}).sort_index(axis = 0,ascending = False)
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py", line 330, in __init__
    mgr = self._init_dict(data, index, columns, dtype=dtype)
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py", line 461, in _init_dict
    return _arrays_to_mgr(arrays, data_names, index, columns, dtype=dtype)
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py", line 6173, in _arrays_to_mgr
    return create_block_manager_from_arrays(arrays, arr_names, axes)
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/internals.py", line 4642, in create_blo

True

## ReportSnapshot: Company overview

In [15]:
snap = pd.DataFrame()
business_summary = None
brief = None
address = None
ratio = pd.DataFrame()
estimate = pd.DataFrame()

tws_conn.register(ReportSnapshot, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='ReportSnapshot')
sleep(5)
tws_conn.unregister(ReportSnapshot, message.fundamentalData)

Server Error: <error id=0, errorCode=430, errorMsg=We are sorry, but fundamentals data for the security specified is not available.Not allowed>


True

# Display Data

In [16]:
snap

""


In [17]:
business_summary

In [18]:
address

In [19]:
quarter.loc[quarter.index[0],'Revenue']

IndexError: index 0 is out of bounds for axis 0 with size 0

Server Error: <error id=0, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>


In [ ]:
int(datetime.strptime(str(quarter.index[0]), "%Y-%m-%d").timestamp())==int(datetime.strptime('20190831', '%Y%m%d').timestamp())

In [16]:
scanner

NameError: name 'scanner' is not defined

In [17]:
annual

""


In [18]:
resc_ann

""


In [19]:
resc_q

""


In [20]:
resc['Sector'][0]

nan

In [21]:
ratio

""


In [22]:
estimate

""


## Technical Data

In [23]:
historical_data = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [29]:
tws_conn.register(historical_data_handler, message.historicalData)
tws_conn.reqHistoricalData(0, contract,'','1 M','1 hour','TRADES',1,1)
sleep(5)
tws_conn.unregister(historical_data_handler, message.historicalData)

True

In [30]:
historical_data['MA30'] = talib.SMA(historical_data['Close'], timeperiod=30)
historical_data['MA10'] = talib.SMA(historical_data['Close'], timeperiod=10)
historical_data['RSI'] = talib.RSI(historical_data['Close'], timeperiod=14)
historical_data['Return'] = (historical_data['Close'] - historical_data['Close'].shift(1))/historical_data['Close']
historical_data['macd'], historical_data['macdsignal'], historical_data['macdhist'] = talib.MACDEXT(historical_data['Close'], fastperiod=12, fastmatype=1, slowperiod=26, slowmatype=1, signalperiod=9, signalmatype=1)
historical_data['Beta'] = talib.BETA(historical_data['High'],historical_data['Low'], timeperiod=5)



In [31]:
historical_data.tail(10)

,Open,High,Low,Close,Volume,MA30,MA10,RSI,Return,macd,macdsignal,macdhist,Beta
Date,,,,,,,,,,,,,
20191010 13:00:00,229.89,229.92,229.10,229.74,16295,227.488000,228.470,64.126358,-0.000696,1.209469,0.985872,0.223597,0.156328
20191010 14:00:00,229.75,229.91,228.30,229.77,26596,227.615333,228.776,64.247424,0.000131,1.219492,1.032596,0.186896,2.048586
20191010 15:00:00,229.77,230.30,229.72,230.09,39154,227.741333,229.073,65.581709,0.001391,1.238974,1.073872,0.165102,2.467842
20191011 09:30:00,232.95,233.82,232.30,233.67,48011,227.967000,229.755,76.257254,0.015321,1.525703,1.164238,0.361465,0.784240
20191011 10:00:00,233.70,234.40,233.66,233.94,45068,228.198000,230.418,76.840688,0.001154,1.754499,1.282290,0.472209,0.712736
20191011 11:00:00,233.95,235.20,233.93,234.48,43546,228.445667,231.166,78.004822,0.002303,1.956838,1.417200,0.539638,0.700329
20191011 12:00:00,234.49,234.87,233.08,234.83,28129,228.648000,231.663,78.750390,0.001490,2.120986,1.557957,0.563029,0.709122
20191011 13:00:00,234.82,235.43,234.47,235.30,26290,228.844333,232.179,79.743372,0.001997,2.262914,1.698948,0.563966,0.735259
20191011 14:00:00,235.31,236.56,234.91,236.49,39656,229.080667,232.821,82.032696,0.005032,2.443252,1.847809,0.595443,0.970188


Server Error: <error id=0, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>


# Step 3: Apply Strategy

# Step 4: Position Account

In [27]:
account = None

In [28]:
tws_conn.register(position_account, message.position)

True

In [29]:
tws_conn.reqPositions()

In [30]:
print(account)

<position account=DU1440139, contract=<ib.ext.Contract.Contract object at 0x7f136554a358>, pos=3500, avgCost=37.59385715>


# Step 5: Placing Orders

In [ ]:
order_id = None

In [ ]:
tws_conn.register(next_id, message.nextValidId)
tws_conn.reqIds(0)

In [26]:
order = create_order(order_type = 'MKT',
                     quantity = 10, 
                     action = 'BUY')

In [27]:
print ('Creating Order: ')
print (' Ticker: ',contract.m_symbol)
print (' SecType: ',contract.m_secType)
print (' Exchange: ',contract.m_exchange)
print (' primExchange: ',contract.m_primaryExch)
print (' Order Quantity: ',order.m_totalQuantity,'\n','Order Type: ', order.m_orderType,'\n','Order Action: ',order.m_action)

Creating Order: 
 Ticker:  AAPL
 SecType:  STK
 Exchange:  SMART
 primExchange:  NASDAQ
 Order Quantity:  10 
 Order Type:  MKT 
 Order Action:  BUY


In [15]:
tws_conn.placeOrder(order_id, contract, order)

Server Error: <error id=62, errorCode=399, errorMsg=Order Message:
BUY 10 AAPL NASDAQ.NMS
Server Error: <error id=62, errorCode=202, errorMsg=Order Canceled - reason:>


# Disconnect

In [20]:
tws_conn.disconnect()

True